3. **Creating a 'customer_activity' dataframe (granularity: user_id)**
   - Aggregate from 'opc_with_dept'; select relevant existing columns
   - Data Transformation - Customer Profiles:
   - 1) reorder_flag: 'adventurous_shopper' vs. 'open_to_suggestion' vs. 'creature_of_habit' 
     2) age_group: 'young_adult' vs. 'middle_age' vs. 'elderly' 
     3) income_level: 'low_income' vs. 'middle_income' vs. 'high_income' 
     4) dietary_preference: 'vegetarian' vs. 'omnivore' vs. 'carnivore' 
     5) food_prep_style: 'non_cook' vs. 'flex_cook' vs. 'enthusiast_cook' 
     6) pet_flag (Boolean)
     7) fam_size: 'single' vs. 'small' vs. 'large' 
     8) shopping_hour_profile: 'early_bird' vs. 'midday_shopper' vs. 'evening_shopper' vs. 'night_owl' 
     9) Export dataframe: 'customer_activity' /update POPULATION FLOW
4. **Creating a 'order_activity' dataframe (granularity: order_id)**
   - Generate 'order_activity' from 'opc_with_dept'
   - Merge with 'customer_activity' dataframe
   - Data Transformtion:
   - 1) Categorize cart size. 
   - Export dataframe / update POPULATION FLOW

Next: 4.10 Part 1d VIZ (visualizations notebook)

In [2]:
# Import libraries. 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
path = r'/Users/amyzhang/Desktop/Instacart Basket Analysis/'

In [4]:
opc_df= pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data 2', 'opc_with_dept.pkl'))

In [5]:
opc_df.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_time', 'days_since_prior_order', 'product_id', 'cart_position',
       'reorder_status', 'product_name', 'department_id', 'prices',
       'delinquent_status', 'price_range_loc', 'day_label',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'avg_order_price',
       'spending_flag', 'median_days_since_prior_order', 'frequency_flag',
       'gender', 'state', 'age', 'date_joined', 'n_dependants', 'fam_status',
       'income', 'region', 'department', 'department_group'],
      dtype='object')

## 3. Aggregate 'customer_df'

In [8]:
# Aggregating by 'user_id' and taking the first value for each of the specified columns
customer_df = opc_df.groupby('user_id')[[
    'date_joined', 'max_order', 'loyalty_flag', 'avg_order_price', 
    'spending_flag', 'median_days_since_prior_order', 'frequency_flag', 
    'gender', 'state', 'region', 'age', 
    'n_dependants', 'fam_status', 'income'
]].first().reset_index()

# Preview the first few rows of the new dataframe
customer_df.head()


,user_id,date_joined,max_order,loyalty_flag,avg_order_price,spending_flag,median_days_since_prior_order,frequency_flag,gender,state,region,age,n_dependants,fam_status,income
0,1,2/17/2019,10,New customer,6.367797,Low spender,20.5,Not frequent customer,Female,Alabama,South,31,3,married,40423
1,10,9/4/2018,5,New customer,8.651049,Low spender,23.0,Not frequent customer,Male,Florida,South,34,2,married,52716
2,100,2/2/2020,5,New customer,9.859259,Low spender,26.0,Not frequent customer,Female,West Virginia,South,64,2,married,138741
3,1000,1/7/2020,7,New customer,8.407767,Low spender,7.0,Frequent customer,Female,Oklahoma,South,25,1,married,63581
4,10000,10/1/2018,72,Loyal customer,8.045421,Low spender,4.0,Frequent customer,Female,Arkansas,South,33,0,single,61257


In [9]:
customer_df.shape

(162633, 15)

#### 162,633 unique customers matches the counts from opc_filtered (after excluding low-activity customers)

## A. Data Transformation: reorder flag

In [12]:
# Group by user_id and calculate total products and total reorders
# as_index=False prevents user_id from becoming the index; it will stay in the df
reorder_activity = opc_df.groupby('user_id', as_index=False).agg(
    total_products=('product_id', 'count'),  # Count total number of products ordered by user
    total_reorders=('reorder_status', 'sum')  # Sum the reorders (1 indicates reorder), user_id
)

# Calculate the reorder proportion for each user
reorder_activity['reorder_proportion'] = reorder_activity['total_reorders'] / reorder_activity['total_products']

# Sort the DataFrame by 'reorder_proportion' in descending order and display the top 20 rows
reorder_activity.sort_values(by='reorder_proportion', ascending=False).head(20)

,user_id,total_products,total_reorders,reorder_proportion
162421,99753,191,189,0.989529
147228,82414,428,420,0.981308
6655,107528,104,102,0.980769
70098,17997,435,426,0.979310
123980,5588,2223,2176,0.978857
61504,170174,47,46,0.978723
103665,3269,275,269,0.978182
17809,12025,45,44,0.977778
154880,91160,212,207,0.976415
74098,184517,210,205,0.976190


In [13]:
reorder_activity.shape

(162633, 4)

In [14]:
# Get the quartiles
reorder_quantiles = reorder_activity['reorder_proportion'].quantile([0.25, 0.5, 0.75])

# Define a function to categorize the users based on their reorder proportion
def categorize_user(row):
    if row['reorder_proportion'] <= reorder_quantiles[0.25]:
        return 'Adventurous Shopper'
    elif row['reorder_proportion'] >= reorder_quantiles[0.75]:
        return 'Creature of Habit'
    else:
        return 'Open to Suggestion'

# Apply the function to create the 'reorder_flag' column
reorder_activity['reorder_flag'] = reorder_activity.apply(categorize_user, axis=1)

# Display the updated DataFrame with 'reorder_flag' for the first few rows
reorder_activity.head()

,user_id,total_products,total_reorders,reorder_proportion,reorder_flag
0,1,59,41,0.694915,Creature of Habit
1,10,143,49,0.342657,Adventurous Shopper
2,100,27,7,0.259259,Adventurous Shopper
3,1000,103,71,0.689320,Creature of Habit
4,10000,1092,836,0.765568,Creature of Habit


In [15]:
# Ensure both 'user_id' columns are of the same type (either int or string)
customer_df['user_id'] = customer_df['user_id'].astype(str)
reorder_activity['user_id'] = reorder_activity['user_id'].astype(str)

# Now, perform the merge
customer_reorder_df = customer_df.merge(reorder_activity, on='user_id', how='left')

In [16]:
customer_reorder_df.head(10)

,user_id,date_joined,max_order,loyalty_flag,avg_order_price,spending_flag,median_days_since_prior_order,frequency_flag,gender,state,region,age,n_dependants,fam_status,income,total_products,total_reorders,reorder_proportion,reorder_flag
0,1,2/17/2019,10,New customer,6.367797,Low spender,20.5,Not frequent customer,Female,Alabama,South,31,3,married,40423,59,41,0.694915,Creature of Habit
1,10,9/4/2018,5,New customer,8.651049,Low spender,23.0,Not frequent customer,Male,Florida,South,34,2,married,52716,143,49,0.342657,Adventurous Shopper
2,100,2/2/2020,5,New customer,9.859259,Low spender,26.0,Not frequent customer,Female,West Virginia,South,64,2,married,138741,27,7,0.259259,Adventurous Shopper
3,1000,1/7/2020,7,New customer,8.407767,Low spender,7.0,Frequent customer,Female,Oklahoma,South,25,1,married,63581,103,71,0.689320,Creature of Habit
4,10000,10/1/2018,72,Loyal customer,8.045421,Low spender,4.0,Frequent customer,Female,Arkansas,South,33,0,single,61257,1092,836,0.765568,Creature of Habit
5,100000,3/26/2018,9,New customer,8.119298,Low spender,30.0,Not frequent customer,Female,Rhode Island,Northeast,52,2,married,108704,114,48,0.421053,Open to Suggestion
6,100001,10/11/2018,66,Loyal customer,7.749815,Low spender,6.0,Frequent customer,Male,South Carolina,South,53,3,married,135036,813,606,0.745387,Creature of Habit
7,100002,9/27/2019,12,Regular customer,6.663462,Low spender,14.0,Regular customer,Male,South Dakota,Midwest,25,0,single,44883,52,26,0.500000,Open to Suggestion
8,100004,8/3/2017,8,New customer,5.823810,Low spender,14.0,Regular customer,Male,Texas,South,46,3,married,44219,42,25,0.595238,Open to Suggestion
9,100005,8/15/2017,18,Regular customer,8.778676,Low spender,12.0,Regular customer,Male,Utah,West,53,3,married,97725,136,68,0.500000,Open to Suggestion


In [17]:
customer_reorder_df.shape

(162633, 19)

## B. Data transformation: age group

In [19]:
customer_reorder_df['age'].describe()

count    162633.000000
mean         49.531528
std          18.475430
min          18.000000
25%          34.000000
50%          49.000000
75%          66.000000
max          81.000000
Name: age, dtype: float64

In [20]:
# Define a function to categorize the users based on their age
def categorize_age_group(age):
    if age <= 34:
        return 'Young Adult'
    elif age <= 64:
        return 'Middle Age'
    else:
        return 'Elderly'

# Apply the function to create the 'age_group' column
customer_reorder_df['age_group'] = customer_reorder_df['age'].apply(categorize_age_group)

In [21]:
# Display the updated DataFrame
customer_reorder_df.head()

,user_id,date_joined,max_order,loyalty_flag,avg_order_price,spending_flag,median_days_since_prior_order,frequency_flag,gender,state,region,age,n_dependants,fam_status,income,total_products,total_reorders,reorder_proportion,reorder_flag,age_group
0,1,2/17/2019,10,New customer,6.367797,Low spender,20.5,Not frequent customer,Female,Alabama,South,31,3,married,40423,59,41,0.694915,Creature of Habit,Young Adult
1,10,9/4/2018,5,New customer,8.651049,Low spender,23.0,Not frequent customer,Male,Florida,South,34,2,married,52716,143,49,0.342657,Adventurous Shopper,Young Adult
2,100,2/2/2020,5,New customer,9.859259,Low spender,26.0,Not frequent customer,Female,West Virginia,South,64,2,married,138741,27,7,0.259259,Adventurous Shopper,Middle Age
3,1000,1/7/2020,7,New customer,8.407767,Low spender,7.0,Frequent customer,Female,Oklahoma,South,25,1,married,63581,103,71,0.689320,Creature of Habit,Young Adult
4,10000,10/1/2018,72,Loyal customer,8.045421,Low spender,4.0,Frequent customer,Female,Arkansas,South,33,0,single,61257,1092,836,0.765568,Creature of Habit,Young Adult


In [22]:
customer_reorder_df['age_group'].value_counts(dropna=False)

age_group
Middle Age     76232
Elderly        43250
Young Adult    43151
Name: count, dtype: int64

In [23]:
total = 76232 + 43250 + 43151
print(total)

162633


## C. Data Transformation: income level

In [25]:
customer_reorder_df['income'].describe()

count    162633.000000
mean      95687.144522
std       42743.513896
min       25903.000000
25%       61718.000000
50%       94292.000000
75%      125165.000000
max      593901.000000
Name: income, dtype: float64

In [26]:
# Get the quantiles (25th, 50th, 75th)
income_quantiles = customer_reorder_df['income'].quantile([0.25, 0.5, 0.75])

# Define the income categories based on quantiles
def categorize_income(income):
    if income <= income_quantiles[0.25]:
        return 'Low Income'
    elif income <= income_quantiles[0.75]:
        return 'Middle Income'
    else:
        return 'High Income'

# Apply the function to create the 'income_group' column
customer_reorder_df['income_group'] = customer_reorder_df['income'].apply(categorize_income)

# Display the updated DataFrame with 'income_group' for the first few rows
customer_reorder_df.head()


,user_id,date_joined,max_order,loyalty_flag,avg_order_price,spending_flag,median_days_since_prior_order,frequency_flag,gender,state,...,age,n_dependants,fam_status,income,total_products,total_reorders,reorder_proportion,reorder_flag,age_group,income_group
0,1,2/17/2019,10,New customer,6.367797,Low spender,20.5,Not frequent customer,Female,Alabama,...,31,3,married,40423,59,41,0.694915,Creature of Habit,Young Adult,Low Income
1,10,9/4/2018,5,New customer,8.651049,Low spender,23.0,Not frequent customer,Male,Florida,...,34,2,married,52716,143,49,0.342657,Adventurous Shopper,Young Adult,Low Income
2,100,2/2/2020,5,New customer,9.859259,Low spender,26.0,Not frequent customer,Female,West Virginia,...,64,2,married,138741,27,7,0.259259,Adventurous Shopper,Middle Age,High Income
3,1000,1/7/2020,7,New customer,8.407767,Low spender,7.0,Frequent customer,Female,Oklahoma,...,25,1,married,63581,103,71,0.689320,Creature of Habit,Young Adult,Middle Income
4,10000,10/1/2018,72,Loyal customer,8.045421,Low spender,4.0,Frequent customer,Female,Arkansas,...,33,0,single,61257,1092,836,0.765568,Creature of Habit,Young Adult,Low Income


In [27]:
customer_reorder_df['income_group'].value_counts(dropna=False)

income_group
Middle Income    81315
Low Income       40660
High Income      40658
Name: count, dtype: int64

In [28]:
total = 81315 + 40660 + 40658
print(total)

162633


## D. Data Transformation: dietary preference

In [30]:
# Identify customers who have never ordered from 'meat seafood' or 'deli'
no_meat_or_deli = opc_df[~opc_df['department'].isin(['meat seafood', 'deli'])]

# Group by 'user_id' and check if they only ordered from departments that are not 'meat seafood' or 'deli'
customers_no_meat_or_deli = no_meat_or_deli.groupby('user_id').size()

# Find customers who have never ordered from these two departments
customers_no_meat_or_deli = customers_no_meat_or_deli[customers_no_meat_or_deli == len(no_meat_or_deli['department'].unique())]

# Count how many customers fall into this category
num_customers_no_meat_or_deli = len(customers_no_meat_or_deli)

# Print the number of customers
print(f"Number of customers who have never ordered from 'meat seafood' or 'deli': {num_customers_no_meat_or_deli}")


Number of customers who have never ordered from 'meat seafood' or 'deli': 728


#### Decision: Abandon Dietary Preference Profile
The initial goal of segmenting customers by dietary preference (vegetarian, omnivore, carnivore) based on department purchases was not viable. The data didn't provide a robust way to define these categories, and only 728 customers had never ordered from meat or seafood departments. As a result, this customer profile for marketing purposes has been abandoned.

## E. Data Transformation: food_prep_style

### Step 1. The count of products in each department for each user. 

In [34]:
# Group by user_id and department, then count the products in each department for each user
product_counts = opc_df.groupby(['user_id', 'department']).size().unstack(fill_value=0)

# Reset the index so that user_id becomes a column
product_counts = product_counts.reset_index()

In [35]:
product_counts.head()

department,user_id,alcohol,babies,bakery,beverages,breakfast,bulk,canned goods,dairy eggs,deli,...,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
0,1,0,0,0,13,3,0,0,13,0,...,2,0,0,0,0,1,0,0,5,22
1,10,0,0,0,0,0,0,2,16,15,...,0,1,13,0,0,22,0,0,72,1
2,100,0,0,2,0,0,0,0,2,5,...,0,0,3,0,0,0,0,0,10,2
3,1000,0,0,1,19,0,0,4,32,3,...,0,0,0,2,0,1,0,0,34,2
4,10000,0,0,56,36,13,1,23,231,47,...,18,2,56,0,0,108,6,0,340,61


### Step 2. Count total products. 

In [37]:
# Count the total number of products per user (sum across all department columns)
department_columns = product_counts.columns.difference(['user_id', 'department'])  # Get department columns
product_counts['total_products'] = product_counts[department_columns].sum(axis=1)


### Step 3. Calculate proportion 

In [39]:
# Calculate proportions for non-cook departments
non_cook_departments = ['frozen', 'bakery', 'canned goods', 'snacks', 'deli']
enthusiast_cook_departments = ['produce', 'dry goods pasta', 'meat seafood', 'pantry', 'dairy eggs']

# Calculate non-cook and enthusiast cook proportions
product_counts['non_cook_proportion'] = product_counts[non_cook_departments].sum(axis=1) / product_counts['total_products']
product_counts['enthusiast_cook_proportion'] = product_counts[enthusiast_cook_departments].sum(axis=1) / product_counts['total_products']

# Check if the columns are correctly added
print(product_counts[['user_id', 'total_products', 'non_cook_proportion', 'enthusiast_cook_proportion']].head())

department user_id  total_products  non_cook_proportion  \
0                1              59             0.372881   
1               10             143             0.132867   
2              100              27             0.444444   
3             1000             103             0.145631   
4            10000            1092             0.222527   

department  enthusiast_cook_proportion  
0                             0.322034  
1                             0.860140  
2                             0.555556  
3                             0.650485  
4                             0.707875  


### Step 4. Calculate difference to create an index. 

In [41]:
# Calculate the difference between the two proportions
product_counts['proportion_difference'] = product_counts['enthusiast_cook_proportion'] - product_counts['non_cook_proportion']

In [42]:
product_counts[['user_id', 'total_products', 'non_cook_proportion', 'enthusiast_cook_proportion', 'proportion_difference']].head(20)

department,user_id,total_products,non_cook_proportion,enthusiast_cook_proportion,proportion_difference
0,1,59,0.372881,0.322034,-0.050847
1,10,143,0.132867,0.860140,0.727273
2,100,27,0.444444,0.555556,0.111111
3,1000,103,0.145631,0.650485,0.504854
4,10000,1092,0.222527,0.707875,0.485348
5,100000,114,0.131579,0.824561,0.692982
6,100001,813,0.134071,0.741697,0.607626
7,100002,52,0.365385,0.461538,0.096154
8,100004,42,0.142857,0.095238,-0.047619
9,100005,136,0.213235,0.375000,0.161765


In [43]:
product_counts['proportion_difference'].describe()

count    162633.000000
mean          0.289110
std           0.291937
min          -1.000000
25%           0.108108
50%           0.310345
75%           0.490566
max           1.000000
Name: proportion_difference, dtype: float64

In [44]:
# Check for NaN values in the entire dataframe
product_counts.isna().sum()

# Check for NaN values specifically in the 'non_cook_proportion' and 'enthusiast_cook_proportion' columns
product_counts[['non_cook_proportion', 'enthusiast_cook_proportion']].isna().sum()


department
non_cook_proportion           0
enthusiast_cook_proportion    0
dtype: int64

In [45]:
# Calculate quantiles for 'proportion_difference'
quantiles = product_counts['proportion_difference'].quantile([0, 0.33, 0.67, 1])

# Define function to classify based on proportion_difference
def classify_prep_style(value):
    if value <= quantiles[0.33]:
        return 'non_cook'
    elif value <= quantiles[0.67]:
        return 'flex_cook'
    else:
        return 'enthusiast_cook'

# Apply classification to the 'proportion_difference' column
product_counts['prep_style'] = product_counts['proportion_difference'].apply(classify_prep_style)

# Check the results
product_counts[['user_id', 'proportion_difference', 'prep_style']].head()


department,user_id,proportion_difference,prep_style
0,1,-0.050847,non_cook
1,10,0.727273,enthusiast_cook
2,100,0.111111,non_cook
3,1000,0.504854,enthusiast_cook
4,10000,0.485348,enthusiast_cook


In [46]:
product_counts['prep_style'].value_counts(dropna=False)

prep_style
flex_cook          55295
non_cook           53669
enthusiast_cook    53669
Name: count, dtype: int64

In [47]:
total = 55295 + 53669 + 53669
print(total)

162633


### Step 5. Merge

In [49]:
# Ensure both 'user_id' columns are of the same type (either int or string)
customer_reorder_df['user_id'] = customer_reorder_df['user_id'].astype(str)
product_counts['user_id'] = product_counts['user_id'].astype(str)

# Now, perform the merge
custom_reorder_prep = customer_reorder_df.merge(product_counts, on='user_id', how='left')

In [50]:
custom_reorder_prep.head()

,user_id,date_joined,max_order,loyalty_flag,avg_order_price,spending_flag,median_days_since_prior_order,frequency_flag,gender,state,...,pantry,personal care,pets,produce,snacks,total_products_y,non_cook_proportion,enthusiast_cook_proportion,proportion_difference,prep_style
0,1,2/17/2019,10,New customer,6.367797,Low spender,20.5,Not frequent customer,Female,Alabama,...,1,0,0,5,22,59,0.372881,0.322034,-0.050847,non_cook
1,10,9/4/2018,5,New customer,8.651049,Low spender,23.0,Not frequent customer,Male,Florida,...,22,0,0,72,1,143,0.132867,0.860140,0.727273,enthusiast_cook
2,100,2/2/2020,5,New customer,9.859259,Low spender,26.0,Not frequent customer,Female,West Virginia,...,0,0,0,10,2,27,0.444444,0.555556,0.111111,non_cook
3,1000,1/7/2020,7,New customer,8.407767,Low spender,7.0,Frequent customer,Female,Oklahoma,...,1,0,0,34,2,103,0.145631,0.650485,0.504854,enthusiast_cook
4,10000,10/1/2018,72,Loyal customer,8.045421,Low spender,4.0,Frequent customer,Female,Arkansas,...,108,6,0,340,61,1092,0.222527,0.707875,0.485348,enthusiast_cook


In [51]:
custom_reorder_prep.shape

(162633, 47)

## F. Data Transformation: pet_flag

In [106]:
# Create the pet_flag column based on whether 'pets' > 0
custom_reorder_prep['pet_flag'] = (custom_reorder_prep['pets'] > 0).astype(int)

In [110]:
custom_reorder_prep[['user_id', 'pets', 'pet_flag']].head()

,user_id,pets,pet_flag
0,1,0,0
1,10,0,0
2,100,0,0
3,1000,0,0
4,10000,0,0


In [114]:
custom_reorder_prep['pet_flag'].value_counts(dropna=False)

pet_flag
0    149458
1     13175
Name: count, dtype: int64

In [116]:
total = 149458 + 13175
print(total)

162633


## G. Data Transformation: family size

In [53]:
def classify_fam_size(n_dependants):
    if 0 < n_dependants <= 2:
        return 'small_fam'
    elif n_dependants >= 3:
        return 'large_fam'
    else:
        return 'single'

In [54]:
custom_reorder_prep['fam_size'] = custom_reorder_prep['n_dependants'].apply(classify_fam_size)

In [55]:
custom_reorder_prep[['user_id', 'n_dependants', 'fam_size']].head()

,user_id,n_dependants,fam_size
0,1,3,large_fam
1,10,2,small_fam
2,100,2,small_fam
3,1000,1,small_fam
4,10000,0,single


In [56]:
custom_reorder_prep.shape

(162633, 48)

## H. Data Transformation: shopping_hour_profile

In [104]:
# 'early_bird' vs. 'midday_shopper' vs. 'evening_shopper' vs. 'night_owl' 

In [125]:
opc_df['order_time'].describe()

count    3.099154e+07
mean     1.341082e+01
std      4.248809e+00
min      0.000000e+00
25%      1.000000e+01
50%      1.300000e+01
75%      1.600000e+01
max      2.300000e+01
Name: order_time, dtype: float64

In [143]:
# Step 1: Find the mode of order_time for each user_id
mode_order_time = opc_df.groupby('user_id')['order_time'].agg(lambda x: x.mode()[0]).reset_index()

# Now 'user_id' is a column, not an index, and we're only including the mode of 'order_time'
mode_order_time.rename(columns={'order_time': 'order_time_mode'}, inplace=True)

# Ensure both 'user_id' columns are of the same type
mode_order_time['user_id'] = mode_order_time['user_id'].astype(str)
custom_reorder_prep['user_id'] = custom_reorder_prep['user_id'].astype(str)

# Now, perform the merge with just the mode
custom_reorder_prep = custom_reorder_prep.merge(mode_order_time[['user_id', 'order_time_mode']], on='user_id', how='left')


In [147]:
custom_reorder_prep[['user_id', 'order_time_mode']].head()

,user_id,order_time_mode
0,1,7
1,10,15
2,100,18
3,1000,18
4,10000,8


In [161]:
custom_reorder_prep.columns

Index(['user_id', 'date_joined', 'max_order', 'loyalty_flag',
       'avg_order_price', 'spending_flag', 'median_days_since_prior_order',
       'frequency_flag', 'gender', 'state', 'region', 'age', 'n_dependants',
       'fam_status', 'income', 'total_products_x', 'total_reorders',
       'reorder_proportion', 'reorder_flag', 'age_group', 'income_group',
       'alcohol', 'babies', 'bakery', 'beverages', 'breakfast', 'bulk',
       'canned goods', 'dairy eggs', 'deli', 'dry goods pasta', 'frozen',
       'household', 'international', 'meat seafood', 'missing', 'other',
       'pantry', 'personal care', 'pets', 'produce', 'snacks',
       'total_products_y', 'non_cook_proportion', 'enthusiast_cook_proportion',
       'proportion_difference', 'prep_style', 'fam_size', 'pet_flag',
       'order_time_mode'],
      dtype='object')

In [163]:
# Define categories based on the mode of order_time
custom_reorder_prep['shopper_hour_profile'] = pd.NA  # Initialize the column

# Categorize based on the mode of order_time
custom_reorder_prep.loc[(custom_reorder_prep['order_time_mode'] >= 0) & (custom_reorder_prep['order_time_mode'] < 6), 'shopper_hour_profile'] = 'night_owl'
custom_reorder_prep.loc[(custom_reorder_prep['order_time_mode'] >= 6) & (custom_reorder_prep['order_time_mode'] < 12), 'shopper_hour_profile'] = 'early_bird'
custom_reorder_prep.loc[(custom_reorder_prep['order_time_mode'] >= 12) & (custom_reorder_prep['order_time_mode'] < 18), 'shopper_hour_profile'] = 'midday_shopper'
custom_reorder_prep.loc[(custom_reorder_prep['order_time_mode'] >= 18) & (custom_reorder_prep['order_time_mode'] < 24), 'shopper_hour_profile'] = 'evening_shopper'


In [165]:
custom_reorder_prep[['user_id', 'order_time_mode', 'shopper_hour_profile']].head()

,user_id,order_time_mode,shopper_hour_profile
0,1,7,early_bird
1,10,15,midday_shopper
2,100,18,evening_shopper
3,1000,18,evening_shopper
4,10000,8,early_bird


In [167]:
custom_reorder_prep.shape

(162633, 51)

## I. Export Dataframe: 'customer_activity'

In [171]:
# Export dataframe. 
custom_reorder_prep.to_pickle(os.path.join(path, '02 Data','Prepared Data 2', 'customer_activity.pkl'))

# 4. Creating order_activity dataframe
### Select columns + derive 'total_products' and 'total_price' for each order

In [176]:
opc_df.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_time', 'days_since_prior_order', 'product_id', 'cart_position',
       'reorder_status', 'product_name', 'department_id', 'prices',
       'delinquent_status', 'price_range_loc', 'day_label',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'avg_order_price',
       'spending_flag', 'median_days_since_prior_order', 'frequency_flag',
       'gender', 'state', 'age', 'date_joined', 'n_dependants', 'fam_status',
       'income', 'region', 'department', 'department_group'],
      dtype='object')

In [178]:
order_activity = opc_df.groupby('order_id').agg({
    'user_id': 'first',                        # Retain the first (or same) value for user_id within each group
    'order_number': 'first',                   # Retain the first value for order_number
    'order_day_of_week': 'first',              # Retain the first value for order_day_of_week
    'order_time': 'first',                     # Retain the first value for order_time
    'days_since_prior_order': 'first',        # Retain the first value for days_since_prior_order
    'day_label': 'first',                       # Retain the first value for day_label
    'busiest_period_of_day': 'first',          # Retain the first value for busiest_period_of_day
    'product_id': 'count',                     # Count the number of product_ids (total products in the order)
    'prices': 'sum'                            # Sum the prices for the order
}).reset_index()

# Rename columns for clarity
order_activity.rename(columns={
    'product_id': 'total_products',
    'prices': 'total_price'
}, inplace=True)



In [180]:
order_activity.head()

,order_id,user_id,order_number,order_day_of_week,order_time,days_since_prior_order,day_label,busiest_period_of_day,total_products,total_price
0,10,135442,4,6,8,8.0,Regularly busy,Average orders,15,132.8
1,1000,49841,7,1,17,5.0,Busiest days,Average orders,7,33.6
2,10000,65685,13,0,11,7.0,Busiest days,Most orders,14,125.7
3,1000000,88772,5,4,17,4.0,Slowest days,Average orders,3,35.1
4,1000001,74636,4,1,23,30.0,Busiest days,Average orders,11,82.2


## A. Data Transformation: categorize cart_size 

In [182]:
# Check the distribution of total_products to decide on the threshold
order_activity['total_products'].describe()

count    3.064545e+06
mean     1.011293e+01
std      7.531930e+00
min      1.000000e+00
25%      5.000000e+00
50%      8.000000e+00
75%      1.400000e+01
max      1.450000e+02
Name: total_products, dtype: float64

In [188]:
# Define cart size thresholds with updated names
def categorize_cart_size(total_products):
    if total_products <= 5:
        return 'small'
    elif 6 <= total_products <= 10:
        return 'moderate'
    elif 11 <= total_products <= 14:
        return 'large'
    else:
        return 'shopping_spree'

# Apply the function to create the 'cart_size' column
order_activity['cart_size'] = order_activity['total_products'].apply(categorize_cart_size)


In [190]:
order_activity[['total_products', 'cart_size']].head()

,total_products,cart_size
0,15,shopping_spree
1,7,moderate
2,14,large
3,3,small
4,11,large


In [194]:
# Ensure both 'user_id' columns are of the same type
order_activity['user_id'] = order_activity['user_id'].astype(str)
custom_reorder_prep['user_id'] = custom_reorder_prep['user_id'].astype(str)

# Perform the LEFT JOIN
order_activity = order_activity.merge(
    custom_reorder_prep[['user_id', 'state', 'region']],
    on='user_id',
    how='left'
)


In [196]:
order_activity.head()

,order_id,user_id,order_number,order_day_of_week,order_time,days_since_prior_order,day_label,busiest_period_of_day,total_products,total_price,cart_size,state,region
0,10,135442,4,6,8,8.0,Regularly busy,Average orders,15,132.8,shopping_spree,Mississippi,South
1,1000,49841,7,1,17,5.0,Busiest days,Average orders,7,33.6,moderate,Illinois,Midwest
2,10000,65685,13,0,11,7.0,Busiest days,Most orders,14,125.7,large,Washington,West
3,1000000,88772,5,4,17,4.0,Slowest days,Average orders,3,35.1,small,Oregon,West
4,1000001,74636,4,1,23,30.0,Busiest days,Average orders,11,82.2,large,Nevada,West


In [198]:
order_activity.shape

(3064545, 13)

In [200]:
opc_df['order_id'].describe()

count     30991542
unique     3064545
top        1564244
freq           145
Name: order_id, dtype: object

## B. Export dataframe. 

In [202]:
# Export dataframe. 
order_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data 2', 'order_activity.pkl'))